In [18]:
import numpy as np
import pandas as pd

from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.utils import to_categorical

# control GPU usage using tensorflow
import tensorflow as tf
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True  # allocate dynamically
session = tf.Session(config=config)


trainRaw =pd.read_csv(r"C:\Users\hari0\Dropbox\Random codes\Movie Reviews\train.tsv",sep='\t')
testRaw =pd.read_csv(r"C:\Users\hari0\Dropbox\Random codes\Movie Reviews\test.tsv",sep='\t')
FinalResults = pd.read_csv(r"C:\Users\hari0\Dropbox\Random codes\Movie Reviews\sampleSubmission.csv",sep=',')
#print(FinalResults)
#FinalResults.drop(['Sentiment'], axis=1)


text = ' '.join(trainRaw.loc[trainRaw.SentenceId == 4, 'Phrase'].values)
text = [i for i in ngrams(text.split(), 3)]

tokenizer = TweetTokenizer()

vectorizer = TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenizer.tokenize)
full_text = list(trainRaw['Phrase'].values) + list(testRaw['Phrase'].values)
vectorizer.fit(full_text)
Ytrain = to_categorical(trainRaw['Sentiment'].values)


embed_size = 300
max_features = 20000
EPOCHS = 20
BATCH_SIZE = 1000
num_classes=Ytrain.shape[1]


tk = Tokenizer(lower = True, filters='')
tk.fit_on_texts(full_text)

train_tokenized = tk.texts_to_sequences(trainRaw['Phrase'])
test_tokenized = tk.texts_to_sequences(testRaw['Phrase'])
print (train_tokenized)

max_len = 50
Xtrain = pad_sequences(train_tokenized, maxlen = max_len)
Xtest = pad_sequences(test_tokenized, maxlen = max_len)

# train, test split
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain, Ytrain, test_size = 0.1)



model=Sequential()
model.add(Embedding(max_features,256,mask_zero=True))
model.add(LSTM(256,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.005),metrics=['accuracy'])
model.summary()

history = model.fit(Xtrain, Ytrain, validation_data=(Xval, Yval),epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

results=model.predict(Xtest)
results = np.round(np.argmax(results, axis=1)).astype(int)
FinalResults['Sentiment'] = results
print (FinalResults)
FinalResults.to_csv("Submission.csv", index=False)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 256)         5120000   
_________________________________________________________________
lstm_11 (LSTM)               (None, None, 256)         525312    
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 645       
Total params: 5,843,077
Trainable params: 5,843,077
Non-trainable params: 0
_________________________________________________________________
Train on 140454 samples, validate on 15606 samples
Epoch 1/20
140454/140454 [==============================] - 45s 323us/step - loss: 1.0039 - acc: 0.5993 - val_loss: 0.8349 - val_acc: 0.6526
Epoch 2/20
140454/140454 [==============================] - 44s 313us/st